In [1]:
import os
from dotenv import load_dotenv

from pydantic import BaseModel, Field

# Langchain imports
from langchain.chat_models import ChatOpenAI

from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain_core.output_parsers import PydanticOutputParser

In [5]:
class Country(BaseModel):
    capital: str = Field(description="capital of the country")
    population: int = Field(description="population of the country")
    name: str = Field (description="name of the country")

PROMPT_COUNTRY_INFO = """Provide information about {country}. If the country is not found make it up.
 {format_instructions}"""

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = "gpt-4"

In [6]:
# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Country)

# setup the chat model
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL)
message = HumanMessagePromptTemplate.from_template(
    template=PROMPT_COUNTRY_INFO,
)
chat_prompt = ChatPromptTemplate.from_messages([message])

# get user input
country_name = "Russia"

# generate the response
print("Generating response...")
chat_prompt_with_values = chat_prompt.format_prompt(
country=country_name, format_instructions=parser.get_format_instructions()
)
output = llm(chat_prompt_with_values.to_messages())
country = parser.parse(output.content)

# print the response
print(f"The capital of {country.name} is {country.capital} and the population is {country.population}.")


/var/folders/d7/kbzjctfd2j1329wsp0mr4g000000gn/T/ipykernel_65141/871088308.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name=OPENAI_MODEL)
/var/folders/d7/kbzjctfd2j1329wsp0mr4g000000gn/T/ipykernel_65141/871088308.py:19: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output = llm(chat_prompt_with_values.to_messages())


Generating response...
The capital of Russia is Moscow and the population is 146599183.


[HumanMessage(content='Provide information about Russia. If the country is not found make it up.\n The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"capital": {"description": "capital of the country", "title": "Capital", "type": "string"}, "population": {"description": "population of the country", "title": "Population", "type": "integer"}, "name": {"description": "name of the country", "title": "Name", "type": "string"}}, "required": ["capital", "population", "name"]}\n```', additional_kwargs={}, response_metadata={})]